In [1]:
# With OHLC (open, high, low ,close)
print("=" * 80)
print("{:=^80}" .format(" With OHLC (open, high, low ,close) "))
print("=" * 80, end="\n\n")

====================== With OHLC (open, high, low ,close) ======================



In [2]:
# Seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

In [3]:
# Warning Message
import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
# Data Loading
import pandas as pd
data = pd.read_csv("./data/df_final.csv")

In [5]:
# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

In [6]:
# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

In [7]:
# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

In [8]:
# 각 종목별 obs 개수
print("="*80)
print("{:=^74}" .format(" 각 종목별 obs 개수 "))
print("="*80)
print(data["code"].value_counts())

============================== 각 종목별 obs 개수 ==============================
000490    733
096870    733
104480    733
138490    733
039570    733
         ... 
287320    494
287330    494
287310    494
570024    494
269620    493
Name: code, Length: 2539, dtype: int64


In [9]:
# 3년간의 데이터가 완전한 종목 코드
print("="*80)
print("{:=^67}" .format(" 3년간 데이터가 완전한 종목 코드 733obs "))
print("="*80)
FULL_DATA_CODE = data["code"].value_counts().index[0:2323]
print(FULL_DATA_CODE)

==================== 3년간 데이터가 완전한 종목 코드 733obs ====================
Index(['000490', '096870', '104480', '138490', '039570', '058730', '039200',
       '200880', '033920', '002140',
       ...
       '215600', '025880', '071460', '101490', '087730', '520006', '071090',
       '006405', '060310', '226350'],
      dtype='object', length=2323)


In [10]:
# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*80)
print("{:=^61}" .format(" 3년간의 데이터가 완전한 종목만 추출한 데이터 "))
print("="*80)
data = data[data["code"].isin(FULL_DATA_CODE)]
print(data)

================= 3년간의 데이터가 완전한 종목만 추출한 데이터 =================
               date   open   high    low  close    code
0        2017-01-02  79905  81585  79170  79800  012320
1        2017-01-03  80535  80535  79170  79800  012320
2        2017-01-04  80535  80955  76965  77175  012320
3        2017-01-05  77070  78855  75600  78225  012320
4        2017-01-06  78225  79800  77280  77385  012320
...             ...    ...    ...    ...    ...     ...
1759974  2019-12-23  14050  15000  13950  14300  215600
1759975  2019-12-24  14400  14400  13500  13650  215600
1759976  2019-12-26  13700  14050  13500  13900  215600
1759977  2019-12-27  13900  14300  13900  14050  215600
1759978  2019-12-30  14250  14750  14150  14550  215600

[1702759 rows x 6 columns]


In [11]:
# RESULT DATASET 생성
FINAL_DATA = pd.DataFrame()

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
temp_data = data                                                                                      
for i in FULL_DATA_CODE :                                                                      
    data = data[data["code"] == i]
    for k in ["open", "high", "low", "close"] : 
        for s in range(1,6):                                                                             
                data['{}{}'.format(k,s)] = data[k].shift(s)                                        
    data = data.dropna().drop(["open", "high", "low", "open1", "open2", "open3", "open4",\
                                "high1", "high2", "high3", "high4",\
                                "low1", "low2", "low3", "low4",\
                                "close1", "close2", "close3", "close4","date", "code"], axis=1)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    data = pd.DataFrame(data, columns = ["close", "open5", "high5", "low5", "close5"])
    FINAL_DATA = FINAL_DATA.append(data)                                                             
    data = temp_data                                                                               
print("="*80)
print("{:=^66}" .format(" SHIFT 생성 : 5일 전 종가로 5일 후 종가를 예측 "))
print("="*80)
FINAL_DATA = FINAL_DATA[["close", "open5", "high5", "low5", "close5"]]
print("DATA Length : {}obs" .format(len(FINAL_DATA)))
print(FINAL_DATA[0:10])

================ SHIFT 생성 : 5일 전 종가로 5일 후 종가를 예측 =================
DATA Length : 1691144obs
      close     open5     high5      low5    close5
0  0.424699  0.419034  0.372414  0.455814  0.414157
1  0.415663  0.413352  0.368276  0.452713  0.415663
2  0.414157  0.413352  0.365517  0.444961  0.408133
3  0.412651  0.403409  0.364138  0.448062  0.408133
4  0.420181  0.401989  0.358621  0.441860  0.408133
5  0.414157  0.407670  0.420690  0.454264  0.424699
6  0.417169  0.423295  0.371034  0.449612  0.415663
7  0.415663  0.413352  0.369655  0.452713  0.414157
8  0.414157  0.417614  0.365517  0.451163  0.412651
9  0.405120  0.416193  0.383448  0.455814  0.420181


In [12]:
# Data 새로 만들기
FINAL_DATA.to_csv("./data/ALLCODE_DATA.csv")

In [13]:
# 새로 만든 데이터 불러오기
data = pd.read_csv("./data/ALLCODE_DATA.csv")

In [14]:
# Feature 추출
data = data[["close", "open5", "high5", "low5", "close5"]]


In [15]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[["open5", "high5", "low5", "close5"]],\
                                                    data[["close"]], test_size=0.10, random_state=42)

In [16]:
# Model Fitting
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs = -1).fit(X_train, y_train)

In [17]:
print("="*80)
print("{:=^80}" .format(" Linear Regression "))
print("="*80)
print("Train Score : {}" .format(reg.score(X_train, y_train)))
print("Test Score : {}" .format(reg.score(X_test, y_test)))

============================== Linear Regression ===============================
Train Score : 0.9275649320317952
Test Score : 0.9280862279204882


In [18]:
# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, reg.predict(X_train))
rmse = mse ** 0.5
print("Train RMSE : {}" .format(rmse))
mse = mean_squared_error(y_test, reg.predict(X_test))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

Train RMSE : 0.06565223126141329
Test RMSE : 0.06519422457786844


In [19]:
# Data Loading
data = pd.read_csv("./data/df_final.csv")

In [20]:
# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

In [21]:
# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))


In [22]:
# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))


In [23]:
# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*80)
print("{:=^76}" .format(" SK 하이닉스 "))
print("="*80)
data = data[data["code"] == "000660"]
print("Data obs : {}" .format(len(data)))
print(data[0:10])

================================= SK 하이닉스 ==================================
Data obs : 733
              date   open   high    low  close    code
833420  2017-01-02  44750  46000  44600  45800  000660
833421  2017-01-03  46200  47300  46200  47250  000660
833422  2017-01-04  47000  47150  46200  46500  000660
833423  2017-01-05  47000  47500  46850  46950  000660
833424  2017-01-06  47700  48450  47600  48000  000660
833425  2017-01-09  48500  49750  48200  49550  000660
833426  2017-01-10  49850  50200  49200  49750  000660
833427  2017-01-11  50000  51900  50000  51600  000660
833428  2017-01-12  51600  51600  50400  50600  000660
833429  2017-01-13  49850  50700  49850  50300  000660


In [24]:
# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
FINAL_DATA = pd.DataFrame()
for k in ["open", "high", "low", "close"] : 
    for s in range(1,6):                                                                             
        data['{}{}'.format(k,s)] = data[k].shift(s)                                        
data = data.dropna().drop(["open", "high", "low", "open1", "open2", "open3", "open4",\
                           "high1", "high2", "high3", "high4",\
                           "low1", "low2", "low3", "low4",\
                           "close1", "close2", "close3", "close4", "date", "code"], axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data, columns = ["close", "open5", "high5", "low5", "close5"])
FINAL_DATA = FINAL_DATA.append(data)                                                             
print("="*80)
print("{:=^76}" .format(" SK 하이닉스 SHIFT DATA "))
print("="*80)
print("DATA Length : {}obs" .format(len(data)))
print(data[0:10])

============================ SK 하이닉스 SHIFT DATA ============================
DATA Length : 728obs
   close     open5     high5   low5    close5
0  0.071  0.000000  0.000000  0.000  0.000000
1  0.075  0.028628  0.025145  0.032  0.029293
2  0.112  0.044423  0.022244  0.032  0.014141
3  0.092  0.044423  0.029014  0.045  0.023232
4  0.086  0.058243  0.047389  0.060  0.044444
5  0.066  0.074038  0.072534  0.072  0.075758
6  0.066  0.100691  0.081238  0.092  0.079798
7  0.057  0.103653  0.114120  0.108  0.117172
8  0.072  0.135242  0.108317  0.116  0.096970
9  0.063  0.100691  0.090909  0.105  0.090909


In [25]:
# 하이닉스 X, y
X = FINAL_DATA[["open5", "high5", "low5", "close5"]].values
y = FINAL_DATA[["close"]].values


In [26]:
# Sk 하이닉스 Score
print("="*80)
print("{:=^76}" .format(" SK하이닉스 Score "))
print("="*80)
print("Test Score : {}" .format(reg.score(X, y)))

=============================== SK하이닉스 Score ===============================
Test Score : 0.9032759294336586


In [27]:
# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, reg.predict(X))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

Test RMSE : 0.07064935296779343


In [39]:
# 경동인베스트 데이터 예측해보기

# Data Loading
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*78)
print("{:=^76}" .format(" DATA "))
print("="*78)
data = data[data["code"] == "032685"]
print("Data obs : {}" .format(len(data)))
print(data[0:10])

print("\n\n")

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
FINAL_DATA = pd.DataFrame()
for k in ["open", "high", "low", "close"] : 
    for s in range(1,6):                                                                             
        data['{}{}'.format(k,s)] = data[k].shift(s)                                        
data = data.dropna().drop(["open", "high", "low", "open1", "open2", "open3", "open4",\
                           "high1", "high2", "high3", "high4",\
                           "low1", "low2", "low3", "low4",\
                           "close1", "close2", "close3", "close4", "date", "code"], axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data, columns = ["close", "open5", "high5", "low5", "close5"])
FINAL_DATA = FINAL_DATA.append(data)                                                             
print("="*78)
print("{:=^76}" .format(" SHIFT DATA "))
print("="*78)
print("DATA Length : {}obs" .format(len(data)))
print(data[0:10])

print("\n\n")

# 하이닉스 X, y
X = FINAL_DATA[["open5", "high5", "low5", "close5"]].values
y = FINAL_DATA[["close"]].values

# Sk 하이닉스 Score
print("="*78)
print("{:=^76}" .format(" DATA Score "))
print("="*78)
print("Test Score : {}" .format(reg.score(X, y)))

# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, reg.predict(X))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

=================================== DATA ===================================
Data obs : 733
              date   open   high    low  close    code
754256  2017-01-02  18000  19100  18000  18600  032685
754257  2017-01-03  18450  19300  18050  19200  032685
754258  2017-01-04  19250  19600  19050  19400  032685
754259  2017-01-05  19550  19600  19150  19500  032685
754260  2017-01-06  19450  19600  18800  19150  032685
754261  2017-01-09  19200  19250  18900  19150  032685
754262  2017-01-10  19150  19150  18300  18500  032685
754263  2017-01-11  18650  24050  18650  24050  032685
754264  2017-01-12  27500  30800  22500  23050  032685
754265  2017-01-13  23000  23300  20750  21700  032685



================================ SHIFT DATA ================================
DATA Length : 728obs
      close     open5     high5      low5    close5
0  0.011818  0.000000  0.000000  0.000000  0.001818
1  0.000000  0.006164  0.002635  0.001131  0.012727
2  0.100909  0.017123  0.006588  0.023756  0.0